# STRAT - Short Tandem Repeat Analysis tool

## 1. Detect flanks and high fidelity reads using alignment

### 1.1 Imports

In [1]:
import gzip
from os import listdir
from os.path import isfile, join

import numpy as np
from string2string.alignment import SmithWaterman

### 1.2 Arguments

In [2]:
prefix = 'AGAAAGAAATGGTCTGTGATCCCCC'
suffix = 'CATTCCCGGCTACAAGGACCCTTCG'
motif = 'CAG'
match_weight = 10  # weight for a match
mismatch_weight = -8  # weight for a mismatch
gap_weight = -9  # weight for a gap
tolerance = 0.2
score_threshold_prefix = len(prefix) * match_weight * (1 - 2*tolerance)
score_threshold_suffix = len(suffix) * match_weight * (1 - 2*tolerance)

# pcr2persons guppy
input_path = '/opt/data/pcr2persons/fastq/guppy/'
output_path = '/opt/data/pcr2persons/output/guppy/'

# pcr2persons dorado
# input_path = '../../../projects/ONT/data/pcr2persons/fastq/dorado/'
# output_path = '../../../projects/ONT/data/pcr2persons/output/dorado/'

# jovan guppy
# input_path = '../../../projects/ONT/data/jovan/fastq/guppy/'
# output_path = '../../../projects/ONT/data/jovan/output/guppy/'

# jovan dorado
# input_path = '../../../projects/ONT/data/jovan/fastq/dorado/'
# output_path = '../../../projects/ONT/data/jovan/output/dorado/'

# dm108 guppy
# input_path = '../../../projects/ONT/data/dm108/fastq/guppy/'
# output_path = '../../../projects/ONT/data/dm108/output/guppy/'

fastq_paths = sorted(join(input_path, f) for f in listdir(input_path) if 'fastq' in f and isfile(join(input_path, f)))
fastq_paths = fastq_paths[:1]
fastq_paths

['/opt/data/pcr2persons/fastq/guppy/AQD087_pass_fae3d762_8343086c_0.fastq.gz']

### 1.3 Constants

In [3]:
DIRECTIONS = ['fwd', 'rev']

COMPLEMENT = {
    'A': 'T',
    'C': 'G',
    'G': 'C',
    'T': 'A'
}

SW = SmithWaterman(
    match_weight=match_weight,  # weight for a match
    mismatch_weight=mismatch_weight,  # weight for a mismatch
    gap_weight=gap_weight,  # weight for a gap
    gap_char=''  # character to use for a gap
)

### 1.4 Functions

In [4]:
def rev_comp(seq, comps):
    return ''.join(comps.get(n, n) for n in reversed(seq))


def chop(string, start, end):
    prefix_flank = string[:start]
    suffix_flank = string[end:]
    ins = string[start:end]
    return prefix_flank, ins, suffix_flank


class ReadProcessor:
    def __init__(
        self,
        sw,
        prefix,
        suffix,
        score_threshold_prefix,
        score_threshold_suffix
    ):
        self.sw = sw
        self.prefix = {
            'fwd': prefix,
            'rev': rev_comp(suffix, COMPLEMENT)
        }
        self.suffix = {
            'fwd': suffix,
            'rev': rev_comp(prefix, COMPLEMENT)
        }
        self.score_threshold_prefix = score_threshold_prefix
        self.score_threshold_suffix = score_threshold_suffix

    def findall(self, target, source, score_threshold):
        aligned_source, aligned_target, score_matrix = self.sw.get_alignment(source, target, return_score_matrix=True)
        scores = np.fromiter((r[-1] for r in score_matrix), float)
        score_max = scores.max()
        ret = None
        
        if score_max > score_threshold:
            score_max_indices = np.where(scores == score_max)
            if len(score_max_indices[0]) == 1:
                # print(aligned_source)
                ret = aligned_source.replace(' | ', '').replace('-', '').replace(' ', '')

        return ret

    def process_read(self, id, seq, opt, qual):
        hits = 0
        row = ['off', id, seq, qual]
        for k in DIRECTIONS:
            prefixes = self.findall(self.prefix[k], seq, self.score_threshold_prefix)
            suffixes = self.findall(self.suffix[k], seq, self.score_threshold_suffix)
            if prefixes and suffixes:
                # print(prefixes, suffixes)
                if hits == 1:
                    hits = 2
                    break
                
                start = seq.index(prefixes) + len(prefixes)
                end = seq.index(suffixes)
                if end > start:
                    direction = k
                    hits = 1
                    prefix_flank, ins, suffix_flank = chop(seq, start, end)
                    prefix_flank_q, ins_q, suffix_flank_q = chop(qual, start, end)

        if hits == 1:
            return [
                'on',
                direction,
                id,
                prefix_flank, ins, suffix_flank,
                prefix_flank_q, ins_q, suffix_flank_q
            ]
        else:
            return row


def process_fastq(fastq_path, output_path, read_processor):
    fastq_name = fastq_path.split('/')[-1]
    gzipped = fastq_name.endswith('.gz')
    hifi_output_path = f'{output_path}{fastq_name}.on-target.tsv'
    # lofi_output_path = f'{output_path}{fastq_name}.off-target.tsv'
    openner = gzip.open if gzipped else open

    with openner(fastq_path, 'rt') as f, open(hifi_output_path, 'wt') as o:
        for i, line in enumerate(f):
            line = line.strip()
            if i%4 == 0:
                if line.startswith('@'):
                    id = line.split(' ')[0]
                else:
                    print(f'Error in {fastq_path} line {i} - not an ID line')
                    raise
            elif i%4 == 1:
                seq = line
            elif i%4 == 2:
                if line.startswith('+'):
                    opt = line
                else:
                    print(f'Error in {fastq_path} line {i} - not a + line')
                    raise
            elif i%4 == 3:
                qual = line
                res = read_processor.process_read(id, seq, opt, qual)
                if res[0] == 'on':
                    o.write('\t'.join(res[1:]) + '\n')
                else:
                    # g.write('\t'.join(res[1:]) + '\n')
                    pass

### 1.5 Main

In [5]:
read_processor = ReadProcessor(SW, prefix, suffix, score_threshold_prefix, score_threshold_suffix)
for fastq_path in fastq_paths:
    print(fastq_path)
    process_fastq(fastq_path, output_path, read_processor)

/opt/data/pcr2persons/fastq/guppy/AQD087_pass_fae3d762_8343086c_0.fastq.gz
